In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv("repoData/margin_stats.csv", index_col=0)
# index to datetime
df.index = pd.to_datetime(df.index)
#rename column
df.replace(np.nan, 0, inplace=True)
df = df.rename(columns={"Debit Balances in Customers' Securities Margin Accounts":"Debit"})
df = df.rename(columns={"Free Credit Balances in Customers' Cash Accounts":"Credit Cash"})
df = df.rename(columns={"Free Credit Balances in Customers' Securities Margin Accounts":"Credit Securities"})
df["Credit"] = df["Credit Cash"] + df["Credit Securities"]
df.drop(["Credit Cash", "Credit Securities"], axis=1, inplace=True)
# calculate Leverage Ratio
df["Leverage Ratio"] = df["Debit"] / df["Credit"]
df["Leverage Ratio"] -= 1.5
df["Leverage Ratio"] = (np.tanh(df["Leverage Ratio"]*2) + 1) *50
# plot the leverage ratio
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(df.index, df["Leverage Ratio"])
plt.title("Leverage Ratio")
plt.xlabel("Date")
plt.ylabel("Leverage Ratio")
plt.show()
# save the leverage ratio
#df.to_csv("repoData/margin_stats.csv")

KeyError: 'Credit Cash'

In [ ]:
import pandas as pd
df = pd.read_csv("repoData/put_call_ratios.csv", parse_dates=["Date"])
df["Date"] = df["Date"].dt.date
df.to_csv("repoData/put_call_ratios.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
def fetch_investor_sentiment_data():
    # Fetch sentiment data from AAII
    # 1. Check for file in data directory
    # TODO validate the dates of the data
    # TODO scrape the recent data from the website (https://www.aaii.com/sentimentsurvey/sent_results)
    date_parser = lambda x: pd.to_datetime(x, format="%m-%d-%Y", errors='coerce')

    aaii_sentiment = pd.read_excel("data/downloads/" + "sentiment.xls", index_col=0, parse_dates=True, date_parser=date_parser)
    # only keep the 6th column
    aaii_sentiment = aaii_sentiment.iloc[:, 5]
    # remove all rows after the first NaN in index column
    aaii_sentiment = aaii_sentiment.iloc[4:pd.Series(aaii_sentiment.index.isna()[4:]).idxmax() + 4]
    aaii_sentiment.ffill(inplace=True)
    aaii_sentiment.replace(np.nan, 0, inplace=True)
    # normalize with tanh
    aaii_sentiment = ((np.tanh(aaii_sentiment * 3) + 1) / 2) * 100
    # save the data
    aaii_sentiment.to_csv("aaii_sentiment.csv")

fetch_investor_sentiment_data()

In [ ]:
import pandas as pd
def diff():
    df = pd.read_csv("backupData/sp500components.csv", index_col=0)
    df_last_components = df.iloc[-1, 0]
    df_last_components = df_last_components.split(",")
    df2 = pd.read_csv("data/sp500_companies.csv")
    df2_last_components = df2.iloc[:, 0].to_list()
    # calculate the difference between the two lists
    if sorted(list(df_last_components)) != sorted(list(set(df_last_components))) or sorted(list(df2_last_components)) != sorted(list(set(df2_last_components))):
        raise ValueError("The input lists must be sets")
    return (list(set(df_last_components) - set(df2_last_components)), list(set(df2_last_components) - set(df_last_components)))
print(diff())
def add_change():
    change = input("Enter a change in the format YYYY-MM-DD;Add1,Add2;Rem1,Rem2:")
    if change.count(";") != 2:
        print("The input must contain two semicolons")
        return
    change = change.split(";")
    change_date = change[0]
    add = change[1].split(",")
    remove = change[2].split(",")
    df = pd.read_csv("backupData/sp500components.csv", index_col=0)
    # last date in the dataframe
    last_date = pd.to_datetime(df.index[-1]).date()
    change_date = pd.to_datetime(change_date).date()
    # if last date is less than the change date
    if last_date < change_date:
        # get the last row in the dataframe
        last_row = df.iloc[-1, 0]
        last_row = last_row.split(",")
        # add the new components
        if add != [""]:
            last_row.extend(add)
        # remove the components
        for i in remove:
            if not i:
                continue
            if i not in last_row:
                raise ValueError("The component to be removed must be in the dataframe")
            last_row.remove(i)
        # remove duplicates
        last_row = sorted(list(set(last_row)))
        # convert the list to a string
        last_row = ",".join(last_row)
        # add the new row to the dataframe
        df.loc[change_date] = last_row
        # save the dataframe
        df.to_csv("backupData/sp500components.csv")

    else:
        raise ValueError("The date must be later than the last date in the dataframe")
add_change()

In [ ]:
import pandas as pd

df = pd.read_csv("backupData/sp500components.csv", index_col=0)
df.index = pd.to_datetime(df.index)
#df = df.resample("D").ffill()
#df = df.iloc[:,0].str.split(",", expand=True)
df

In [ ]:
# import os
# import utils
# data_dir = "test_data/"
# if not os.path.exists(data_dir):
#     os.mkdir(data_dir)

In [ ]:
utils.fetch_sp500companies_data(data_dir, sp500_dir, "2019-01-01", "2020-01-02")

# Test Metrics

In [1]:
import os
from metrics import *
import matplotlib
matplotlib.use('tkagg')
import plotting
data_dir = "test_data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
start_date = '2000-01-01'
end_date = None#'2024-01-01'

In [2]:
def showMetricsCalculation(metric : Metric, y_axis = ["left", "left", "right"]):
    if False:
        sp500 = SP500Momentum()
        sp500.get()
    # plot all stages (data, processed, result)
    graph_list = [
        plotting.Graph(metric.data, y_axis=y_axis[0], df_color="black", df_label="data"), 
        plotting.Graph(metric.processed, y_axis=y_axis[1], df_color="blue", df_label="processed"), 
        plotting.Graph(metric.result, y_axis=y_axis[2], df_color="red", df_label="result"),
        #plotting.Graph(sp500.data, y_axis=y_axis[0], df_color="green", df_label="train"),
        #plotting.Graph(metric.test, y_axis=y_axis[2], df_color="orange", df_label="test")
    ]
    plotting.plot_graph(graph_list, neutral_line=None, title=metric.name)

In [3]:
Metric.setPreferences(data_dir, start_date, end_date)
metrics = []
metrics.append(SaveHavenDemand())
metrics.append(ConsumerSentiment())
metrics.append(SP500Momentum())
metrics.append(PutCallRatio())
metrics.append(InsiderTransactions())
metrics.append(AAIISentiment())
metrics.append(MarginStats())
metrics.append(VIX())
for metric in metrics:
    metric.get()
    showMetricsCalculation(metric)
    break